In [2]:
import sys
import os
from os import path
from os import walk

import re

import pandas as pd
import json
import logging
import pandas_profiling
from pandas.io.json import json_normalize 

In [153]:
class FileIterator:
    list = [".json~", ".swp" ,".DS_Store"]

    def __init__(self, input_root, input_folders):
        self.input_data_dir = input_root
        self.input_folders = input_folders
        
    def iterate_filenames(self):
        resources = self.__iterate_dirfiles()
        files = self.__iterate_file(resources)
        return files
        
    def __iterate_dirfiles(self):
        path_files = []
        file_names = self.input_folders

        for file in file_names:
            path_files.append(input_data_dir + '/' + file)
        return path_files
    
    def __iterate_file(self, path_dbs):
        path_db_workload_files = []     
        for path_db in path_dbs:
            for root, dirs, files in walk(path_db):  
                for filename in files:
                    if not filename.endswith(('.json~', '.swp')):
                        print('valid file: ',filename)
                        path_db_workload_files.append(root + '/' + filename)
                    if ''.join(dirs) != '.ipynb_checkpoints':
                        pass
                    
        return path_db_workload_files

In [104]:
class ContentReader():
    
    def __init__(self, filename_paths):
        self.filename_paths = filename_paths
        
    def get_data(self):
        data = []
        for filename_path in self.filename_paths:
            
            json = ReadJSON()
            data.append(json.read(filename_path))

        return data

In [103]:
class ReadJSON():
    
    @staticmethod
    def read(file_path):
        with open(file_path) as f:
            d = json.load(f)
        return d

In [144]:
class RedisImport():
    
    @staticmethod
    def insert_all(data):
        
        for item in data:
            key = item['resourceType'] + ':' + item['id']
            
            try:
                r = redis.Redis(host='localhost', port=6379, db=0)
                #print(key)
                result = json.loads(r.execute_command('JSON.SET', key, '.', json.dumps(item)))
            except Exception as e:
                #print(key)
                result = e
        return result
                

In [155]:
root_dir = '../'

input_data_folders = ['Patient','Condition']
input_data_dir = root_dir + 'output/fhir-json-redis'

files = FileIterator(input_data_dir, input_data_folders)
input_dirpaths = files.iterate_filenames()

data = []
files = ContentReader(input_dirpaths)
data = files.get_data()

result = RedisImport.insert_all(data)

valid file:  0.Patient.json
valid file:  1.Patient.json
valid file:  0.Condition.json
valid file:  1.Condition.json
